<a href="https://colab.research.google.com/github/higherbar-ai/ai-workflows/blob/main/example-google-colab-document-processing.ipynb" target="_parent"><img alt="Open In Colab" src="https://colab.research.google.com/assets/colab-badge.svg"/></a>

# About this notebook

This notebook provides a simple example of how to use the `ai-workflows` package in Google Colab.

## Configuration

This notebook uses secrets, which you can configure by clicking on the key icon in Google Colab's left sidebar.

This notebook requires access to OpenAI models. To use OpenAI directly, configure the `openai_api_key` secret to contain your API key. (Get a key from [the OpenAI API key page](https://platform.openai.com/api-keys), and be sure to fund your platform account with at least $5 to allow GPT-4o model access.)

Alternatively, you can use OpenAI via Microsoft Azure by configuring the following secrets:

1. `azure_api_key`
2. `azure_api_base`
3. `azure_api_engine`
4. `azure_api_version`

Finally, you can override the default model of `gpt-4o` by setting the `openai_model` secret to your preferred model, and you can optionally add [LangSmith tracing](https://langchain.com/langsmith) by setting the `langsmith_api_key` secret.

## Setting up the runtime environment

The next code block will install everything necessary to set up the runtime environment for this notebook.

In [ ]:
!pip install google-colab
!pip install py-ai-workflows

# download NLTK data
import nltk
nltk.download('punkt', force=True)

# install LibreOffice for document processing
!apt-get install -y libreoffice

## Initializing

The next code block initializes the notebook by loading secrets and initializing the LLM interface.

In [ ]:
from google.colab import userdata
from ai_workflows.llm_utilities import LLMInterface
from ai_workflows.document_utilities import DocumentInterface

# utility function: get secret from Google Colab, with support for a default
def get_secret_with_default(secretName, defaultValue=None):
    # noinspection PyBroadException
    try:
        return userdata.get(secretName)
    except:
        return defaultValue


# read all supported secrets
openai_api_key = get_secret_with_default('openai_api_key')
openai_model = get_secret_with_default('openai_model', 'gpt-4o')
azure_api_key = get_secret_with_default('azure_api_key')
azure_api_base = get_secret_with_default('azure_api_base')
azure_api_engine = get_secret_with_default('azure_api_engine')
azure_api_version = get_secret_with_default('azure_api_version')
langsmith_api_key = get_secret_with_default('langsmith_api_key')

# complain if we don't have the bare minimum to run
if not openai_api_key and not (azure_api_key
                               and azure_api_base
                               and azure_api_engine
                               and azure_api_version):
    raise Exception('We need either an openai_api_key secret set in the secrets — or set azure_api_key, azure_api_base, azure_api_engine, and azure_api_version to use Azure instead. Also be sure to enable Notebook Access for the secret(s).')

# initialize LLM interface
llm = LLMInterface(openai_api_key=openai_api_key, openai_model=openai_model, azure_api_key=azure_api_key, azure_api_base=azure_api_base, azure_api_engine=azure_api_engine, azure_api_version=azure_api_version, temperature = 0.0, total_response_timeout_seconds=600, number_of_retries=2, seconds_between_retries=5, langsmith_api_key=langsmith_api_key)

# initialize our document processor
doc_interface = DocumentInterface(llm_interface=llm)

# set Google Colab content directory
content_dir = "/content"

# report success
print("Initialization successful.")

## Document upload

This next code block prompts you to upload one or more files to convert to markdown, then it converts them and outputs them in the same Colab content directory. (Find, view, and download all files by clicking on the folder icon in the left sidebar.)

In [ ]:
from google.colab import files
import os

# prompt for one or more files
filename = None
print('Upload one or more files to convert to Markdown:')
print()
uploaded = files.upload()

# if we got any files, process them one by one
if len(uploaded.items()) > 0:
    # loop through all uploaded files
    for filename in uploaded.keys():
        print(f'Processing {filename}...')
        file_path = os.path.join(content_dir, filename)
        
        # convert to markdown
        markdown = doc_interface.convert_to_markdown(file_path)

        # write the markdown to the output directory
        output_path = os.path.join(content_dir, os.path.splitext(os.path.basename(filename))[0] + '.md')
        with open(output_path, 'w') as f:
            f.write(markdown)

        print(f"Conversion complete. Markdown saved to {output_path}")